In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
from PyAstronomy import pyasl
import hpfspec2
import glob
import random
import datetime
import ipywidgets as widget
import ipympl
import os
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.patches as patches
from PIL import Image
%matplotlib inline

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Multiple Plots (test)

In [2]:
dataToPull = ['DataToPlot_Etalon','DataToPlot_LFC','DataToPlot_Etalon_c2_sQ','DataToPlot_LFC_c2_sQ']

In [3]:
#if we want to pull all the files for each system/directory, match the channels, and plot them together
saved_data_0 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+dataToPull[0]+'/*.csv'))
saved_data_1 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+dataToPull[1]+'/*.csv'))
saved_data_2 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+dataToPull[2]+'/*.csv'))
saved_data_3 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+dataToPull[3]+'/*.csv'))

master_list_etalon = []
for i in range(28):
    def modeNum(curFile):
        return int(curFile[76:][curFile[76:].find('ModesAt')+7:-4])
    curOrder = []
    for eachFile in saved_data_0:
        if eachFile[74]=='_':
            corrected_str_part1 = 'FullRawData_1OrdersAt0'+eachFile[73]+'_'
        else:
            corrected_str_part1 = eachFile[52:76]
        if int(corrected_str_part1[-3:-1])==i:
            curOrder.append(eachFile)
    curOrder.sort(key=modeNum)
    master_list_etalon.append(curOrder)

#same as Etalon except 6 more letters in name
master_list_etalon_c2_sQ = []
for i in range(28):
    def modeNum(curFile):
        return int(curFile[82:][curFile[82:].find('ModesAt')+7:-4])
    curOrder = []
    for eachFile in saved_data_2:
        if eachFile[80]=='_':
            corrected_str_part1 = 'FullRawData_1OrdersAt0'+eachFile[79]+'_'
        else:
            corrected_str_part1 = eachFile[58:82]
        if int(corrected_str_part1[-3:-1])==i:
            curOrder.append(eachFile)
    curOrder.sort(key=modeNum)
    master_list_etalon_c2_sQ.append(curOrder)

# 3 less letters than etalon
master_list_LFC = []
for i in range(28):
    def modeNum(curFile):
        return int(curFile[73:][curFile[73:].find('ModesAt')+7:-4])
    curOrder = []
    for eachFile in saved_data_1:
        if eachFile[71]=='_':
            corrected_str_part1 = 'FullRawData_1OrdersAt0'+eachFile[70]+'_'
        else:
            corrected_str_part1 = eachFile[49:73]
        if int(corrected_str_part1[-3:-1])==i:
            curOrder.append(eachFile)
    curOrder.sort(key=modeNum)
    master_list_LFC.append(curOrder)

# 6 more letters than LFC
master_list_LFC_c2_sQ = []
for i in range(28):
    def modeNum(curFile):
        return int(curFile[79:][curFile[79:].find('ModesAt')+7:-4])
    curOrder = []
    for eachFile in saved_data_3:

        if eachFile[77]=='_':
            corrected_str_part1 = 'FullRawData_1OrdersAt0'+eachFile[76]+'_'
        else:
            corrected_str_part1 = eachFile[53:79]
        
        if int(corrected_str_part1[-3:-1])==i:
            curOrder.append(eachFile)
    curOrder.sort(key=modeNum)
    master_list_LFC_c2_sQ.append(curOrder)

    
saved_data = []
for i in range(28):
    for j in range(8):
        saved_data.append([master_list_LFC_c2_sQ[i][j],master_list_LFC[i][j],master_list_etalon_c2_sQ[i][j],master_list_etalon[i][j]])


In [ ]:
data_to_plot = ['/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon/Other/FullRawData_All_Orders_All_Modes.csv','/data4/HPF/etalon/data/DataToPlot/DataToPlot_LFC/Other/FullRawData_All_Orders_All_Modes.csv','/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon_c2_sQ/Other/FullRawData_All_Orders_All_Modes.csv']

In [5]:
saved_data = [['/data4/HPF/etalon/data/DataToPlot/DataToPlot_LFC_c2_sQ/Other/FullRawData_All_Orders_All_Modes.csv','/data4/HPF/etalon/data/DataToPlot/DataToPlot_LFC/Other/FullRawData_All_Orders_All_Modes.csv','/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon_c2_sQ/Other/FullRawData_All_Orders_All_Modes.csv','/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon/Other/FullRawData_All_Orders_All_Modes.csv']]

In [6]:
data_to_plot = saved_data[0]
print(data_to_plot)

['/data4/HPF/etalon/data/DataToPlot/DataToPlot_LFC_c2_sQ/Other/FullRawData_All_Orders_All_Modes.csv', '/data4/HPF/etalon/data/DataToPlot/DataToPlot_LFC/Other/FullRawData_All_Orders_All_Modes.csv', '/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon_c2_sQ/Other/FullRawData_All_Orders_All_Modes.csv', '/data4/HPF/etalon/data/DataToPlot/DataToPlot_Etalon/Other/FullRawData_All_Orders_All_Modes.csv']


In [39]:
plt.close('all') #we are generating alot of figs so make sure any leftovers are gone

#SETTINGS BEGIN

AllPlots = True #if we want all the channels in every order
#otherwise split it by splitting the array created above
splitIndex_head = 0
splitIndex_tail = 1

#SETTINGS END

if AllPlots:
    plottableData = saved_data
else:
    plottableData = saved_data[splitIndex_head:splitIndex_tail]

for number,data_to_plot in enumerate(plottableData): #loop over master list to pull each set of files

    
#SETTINGS BEGIN
    saveFigures = True #do we want to save the figures automatically
    StorageSpace = 'Other' #what directory in /data4/HPF/etalon/data/Figures/ do we want to save to
    limit = True #limit the y-axis if we are saving the fig
    limit_nums = [-15,15] #left is lower, right is upper
    print_key = False #if we want to print the order/mode location together with pixel location under graph


    plot_velocity = True #do we want to plot velocity shifts (overwrites fwhm)
    plot_fwhm = True #do we want to plot fwhm
    plot_pixels = True #plot pixels (FALSE IS WAVELENGTH)
    plotTrendLines = False #if we want to plot a rough linear line-of-best-fit
    complexLabel = True #if we want the info on the plot
    account_for_offset = False #do we want center data around 0 (only important with FWHM)
    subplot = True #danger not entirely implemented so may or may not mess things up
    #its supposed to help transition between plotting as one plot or plotting as subplots or something like that

#SETTINGS END
    
    #initialize data
    fitsettings_array = []
    key = {}
    channel_num = number%8
    origin_x = 85
    origin_y = 47
    length = 45
    height = 15
    x_increment = 45
    y_increment = 18.5
    

    
    for myNum,eachFile in enumerate(data_to_plot):
        #pull the data stored in csv (specific format to pull from)
        df_in = pd.read_csv(eachFile)
        master_list = df_in.transpose().to_numpy()
        times = pd.to_datetime(master_list[1]) 
        c_names = df_in.columns #keep track what data we are pulling (i.e. Wavelength, Centroid)

        centroidsWL = []
        for num in master_list[2]:
            centroidsWL.append(float(num))

        fwhmWL = []
        for num in master_list[3]:
            fwhmWL.append(float(num))

        centroidsPIX = []
        for num in master_list[4]:
            centroidsPIX.append(float(num))

        fwhmPIX = []
        for num in master_list[5]:
            fwhmPIX.append(float(num))

        v_list = []
        for num in master_list[6]:
            v_list.append(float(num))

    #PLOT
        XLABEL = c_names[1] #label x-axis of time
        AVG = c_names[2][:-12] #pull the average method we are using
        Title = AVG+' Mode Position over Time' #create title
        
        #to label the the location which we are looking at
        avg_location_wl = np.nanmedian(centroidsWL) 
        avg_location_pix = np.nanmedian(centroidsPIX)

        #LABELS
        #depends on settings 
        if plot_velocity:
                YLABEL = 'Velocity Shift [m/s]'
                Title = AVG+ 'Velocity Shift over Time'
                style = 'o'
                array = v_list
                time = times
        elif plot_fwhm:
            YLABEL = 'FWHM [Å]'
            Title = AVG+ 'FWHM over Time'

            if plot_pixels: #FWHM_Pixel
                YLABEL = AVG+'FWHM [pixels]'
                Title = AVG+'FWHM over Time'
                style = 'P'
                fwhmToPlot = fwhmPIX
            else: #wavelength
                YLABEL = AVG+'FWHM [Å]'
                Title = AVG+'FWHM over Time'
                style = 'o'
                fwhmToPlot = fwhmWL

            time = times
            array = fwhmToPlot
        else:
            if plot_pixels:
                YLABEL = AVG+'Mode Position [pixels]'
                Title = AVG+'Mode Position over Time'
                style = 'P'
                centroidToPlot = centroidsPIX
            else: #wavelength
                YLABEL = AVG+'Mode Position [Å]'
                Title = AVG+'Mode Position over Time'
                style = 'o'
                centroidToPlot = centroidsWL

            time = times
            array = centroidToPlot

        #pull what orders/modes we are looking at from file name
        #SPECIFIC INDEXING OF NAME, IF PATHS OF FILES ARE CHANGED, THESE INDICES WILL ALSO CHANGE
        if eachFile[34:54] == 'DataToPlot_LFC_c2_sQ':
            fitsettings = eachFile[67:-4]
            if subplot:
                curColor = 'tab:orange'
                calibMode = 'LFC '
            else:
                curColor = 'tab:red'
                calibMode = 'LFC_c2_sQ '
            curSubplot = True #filter for which plot it should be on

        elif eachFile[34:48] == 'DataToPlot_LFC':
            fitsettings = eachFile[61:-4]
            calibMode = 'LFC '
            curColor = 'tab:orange'
            curSubplot = False

        elif eachFile[34:57] == 'DataToPlot_Etalon_c2_sQ':
            fitsettings = eachFile[70:-4]
            if subplot:
                curColor = 'tab:blue'
                calibMode = 'Etalon '
            else:
                curColor = 'tab:green'
                calibMode = 'Etalon_c2_sQ '
            curSubplot = True

        elif eachFile[34:51] == 'DataToPlot_Etalon':
            fitsettings = eachFile[64:-4]
            calibMode = 'Etalon '
            curColor = 'tab:blue'
            curSubplot = False


        else:
            fitsettings = 'incorrect indexing'
            calibMode = 'incorrect indexing'
        
        #create a key to keep track of the fitsettings AND the avg_location
        key[fitsettings] = [avg_location_wl,avg_location_pix]

        fitsettings_array.append(fitsettings)
        
        #pull out information
        IndexOrder = fitsettings.find('rsAt')+4
        IndexMode = fitsettings.find('Mo')-1
        
        #orders
        if fitsettings == 'wData_All_Orders_All_Modes':
            num_mode_str = 'all'
            mode_loc_str = 'all'
            order_loc_str = 'all'
        else:
            if fitsettings[IndexOrder+1] == '_':
                order_loc_str = fitsettings[IndexOrder]
            else:
                order_loc_str = fitsettings[IndexOrder:IndexOrder+2]
            #modes
            if fitsettings[IndexMode-1] == '_':
                num_mode_str = fitsettings[IndexMode]
            elif fitsettings[IndexMode-2] == '_':
                num_mode_str = fitsettings[IndexMode-1:IndexMode+1]
            else:
                num_mode_str = fitsettings[IndexMode-2:IndexMode+1]
            mode_loc_str = fitsettings[IndexMode+8:]

            curOrd = int(order_loc_str)
            #set label that is next to graph (clean or messy way)
        if complexLabel:   
            FileInformation = 'Order: '+order_loc_str+'\nModes: '+num_mode_str+'\nPixel: '+str(round(avg_location_pix))+'\nWavelength: '+str(round(avg_location_wl))
        else:
            FileInformation = key
        
        
        #center around 0
        if account_for_offset:
            plotTitle = Title + ' centered at 0'
            to_plot = array - np.nanmedian(array)
        else:
            plotTitle = Title
            to_plot = array
                
        #create the figure and subplots
        if myNum==0:
            fig = plt.figure(num=fitsettings_array[0],figsize=(13,6),constrained_layout=True)
            ax1 = fig.add_subplot(221, title='Normal Pipeline')
            ax2 = fig.add_subplot(223,title='sQ Pipeline', sharex=ax1)
            ax3 = fig.add_subplot(122)
            ax3.axis('off')
            fig.suptitle(plotTitle, fontsize=17)

        
        #plot the data
        if curSubplot:
            pts2 = ax2.plot(time, to_plot, style, color = curColor, label = calibMode, alpha = 0.4, ms=2)
            ax2.set_ylabel(YLABEL)
            ax2.set_title('sQ Pipeline',size=14)
            ax2.grid(True)

        else:
            pts1 = ax1.plot(time, to_plot, style, color = curColor, label = calibMode, alpha = 0.4, ms=2)
            ax1.set_ylabel(YLABEL)
            ax1.set_title('Normal Pipeline',size=14)
            ax1.grid(True)

        ax2.set_xlabel(XLABEL)
        
        #if we want a generic trendline with the points
        if plotTrendLines:
            # calc the trendline
            x = mdates.date2num(time)
            z = np.polyfit(x, to_plot, 1)
            p = np.poly1d(z)
            #plot the trendline
            if curSubplot:
                ax2.plot(time,p(x),"k--")
            else:
                ax1.plot(time,p(x),"k--")
            # the line equation:
            print('y=%.6fx+(%.6f)'%(z[0],z[1]))
            #if we want to implement displaying the slope like Ryan's plots
            #slope = 0 
            #text = str(slope)
            #plt.text(x[-1]+100, p[-1],text, fontsize=14)
    
    #pull ax2 label to use on ax3
    lines, labels = ax2.get_legend_handles_labels()
    
    #create the master legend
    leg = ax3.legend(lines, labels,title='Source', bbox_to_anchor=(-0.2,0.3), loc=6,fontsize = 'large',shadow=True)
    
    #darken the colors in the label so we can see them but keep the actual points light
    for lh in leg.legendHandles:
        lh._legmarker.set_alpha(1)
    
    #if we want an automatic y-limit according to the data
    stdev = 3* np.std(to_plot)
    ylimits = [-stdev,stdev]
    #if we want a designated limit
    if limit:
        ylimits = limit_nums   
    ax1.set_ylim(ylimits)
    ax2.set_ylim(ylimits)
    
    # Display the spectrograph
    im = Image.open('/data4/HPF/etalon/data/Figures/Other/Etalon Legend.png')
    ax3.imshow(im)
    
    # Create a Rectangle patch to mark the spot of the spectrograph
    #WARNING THIS DEPENDS ON THE ORDER THE FILES ARE COMING IN
    if fitsettings == 'wData_All_Orders_All_Modes':
        rect = patches.Rectangle((origin_x, origin_y), length+325, height+515, linewidth=1, edgecolor='r', facecolor='none')
    else:
        rect = patches.Rectangle((origin_x+(channel_num*x_increment), origin_y+(curOrd*y_increment)), length, height, linewidth=1, edgecolor='r', facecolor='none')
    
    # Add the patch to the spectrograph
    ax3.add_patch(rect)
    
    #add text to display information
    ax3.text(-100, 200,FileInformation, fontsize=11, clip_on=False,backgroundcolor='gray')
    
    #save the figure with proper format in the designated space
    if saveFigures:
        if fitsettings == 'wData_All_Orders_All_Modes':
            plt.savefig('/data4/HPF/etalon/data/Figures/'+StorageSpace+'/AllOrders_AllModes.png')
        else:
            ordName = 'Order'    
            if curOrd<10:
                ordName = 'Order0'
            plt.savefig('/data4/HPF/etalon/data/Figures/'+StorageSpace+'/'+ordName+str(curOrd)+'_'+'Channel_'+str(1+channel_num)+'.png')
    
    if print_key:
        print(key)
      
    plt.close('all') #close figs  

In [5]:
#NOT QUITE WORKING
#this cell pulls all the files from each of the above directories that have the same basename
saved_data = []

temparray_0 = []
master_temparray = []
saved_data_0 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+dataToPull[0]+'/*.csv'))
for k,data in enumerate(dataToPull):
    temparray = []
    saved_data_1 = sorted(glob.glob('/data4/HPF/etalon/data/DataToPlot/'+data+'/*.csv'))
    for eachFile in saved_data_0:
        compare_file_1 = '/data4/HPF/etalon/data/DataToPlot/'+data+'/'+os.path.basename(eachFile)
        if compare_file_1 in saved_data_1:
            temparray.append(compare_file_1)
            temparray_0.append(eachFile)
    if k==0:
        master_temparray.append(temparray_0)
    master_temparray.append(temparray)
    
#print(len(master_temparray[]))
saved_data = [[master_temparray[i][j] for i in range(len(dataToPull))] for j in range(len(master_temparray[0]))]
print(saved_data)

IndexError: list index out of range